# Step 1: Problem Definition and Requirements

**Objective:**  
Predict travel time (duration in seconds) for ride trips given start and end locations (specified in WGS84 coordinates) and a departure datetime.

**Datasets:**  
- **train.csv:** Contains columns:
  - `row_id`, `start_lng`, `start_lat`, `end_lng`, `end_lat`, `datetime`, `duration`
- **test.csv:** Contains columns:
  - `row_id`, `start_lng`, `start_lat`, `end_lng`, `end_lat`, `datetime`

**Evaluation Metric:**  
Root Mean Squared Error (RMSE) computed as:  
```python
RMSE = sqrt(mean_squared_error(prediction, ground_truth))

# 2. Data Ingestion and Exploration

- Load train.csv and test.csv; inspect data types, distributions, and anomalies.
- Explore geospatial distributions, temporal trends, and basic summary statistics.
- Justification: Early data insights help identify quality issues, outliers, and potential transformations essential for robust modeling.